In [1]:
import pandas as pd
import requests
from tqdm import tqdm_gui, tqdm
import time
import os
import sys

from src import parsing, utils, config
import functools

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
my_api_key = 'a5b199a5-0641-4d03-8cf8-2df1e19e52ef'

# Collect latest listings
- Top 100
- Ordered by Market Capitalisation
- Source: https://pro-api.coinmarketcap.com

In [3]:
listings = parsing.get_listings(
    url         = config.latest_listings_url,
    headers     = {'X-CMC_PRO_API_KEY' : my_api_key},
    sortedby    = 'market_cap',
    limit       = 100
    )

listings

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote
0,1,Bitcoin,BTC,bitcoin,11013,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.000,19688100.000,19688100.000,False,None,1,NaN,NaN,NaN,2024-04-21T12:08:00.000Z,"{'USD': {'price': 65299.253514800956, 'volume_..."
1,1027,Ethereum,ETH,ethereum,8845,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,120071333.213,120071333.213,True,None,2,NaN,NaN,NaN,2024-04-21T12:07:00.000Z,"{'USD': {'price': 3172.4229401151097, 'volume_..."
2,825,Tether USDt,USDT,tether,83590,2015-02-25T00:00:00.000Z,"[payments, stablecoin, asset-backed-stablecoin...",NaN,109834251114.814,112941048614.413,True,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",3,NaN,NaN,NaN,2024-04-21T12:07:00.000Z,"{'USD': {'price': 1.00020915287623, 'volume_24..."
3,1839,BNB,BNB,bnb,2151,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",NaN,149533165.231,149533165.231,False,None,4,NaN,NaN,NaN,2024-04-21T12:07:00.000Z,"{'USD': {'price': 578.8311243476501, 'volume_2..."
4,5426,Solana,SOL,solana,655,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,446930668.693,574447174.319,True,None,5,NaN,NaN,NaN,2024-04-21T12:07:00.000Z,"{'USD': {'price': 151.50704902562038, 'volume_..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1765,EOS,EOS,eos,665,2017-07-01T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, sma...",NaN,1123641235.035,1123641235.035,True,None,96,1123645555.648,927159559.533,NaN,2024-04-21T12:07:00.000Z,"{'USD': {'price': 0.8251352527249441, 'volume_..."
96,2087,KuCoin Token,KCS,kucoin-token,47,2017-10-24T00:00:00.000Z,"[marketplace, centralized-exchange, discount-t...",170118638.000,96132689.289,143632689.288,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",97,NaN,NaN,NaN,2024-04-21T12:08:00.000Z,"{'USD': {'price': 9.548808598787874, 'volume_2..."
97,1966,Decentraland,MANA,decentraland,491,2017-09-17T00:00:00.000Z,"[platform, collectibles-nfts, gaming, payments...",NaN,1908319865.392,2193179327.320,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",98,NaN,NaN,NaN,2024-04-21T12:07:00.000Z,"{'USD': {'price': 0.4681192792405877, 'volume_..."
98,18876,ApeCoin,APE,apecoin-ape,400,2022-03-17T12:26:36.000Z,"[collectibles-nfts, content-creation, gaming, ...",1000000000.000,604895833.000,1000000000.000,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",99,604895833.000,793164122.682,NaN,2024-04-21T12:08:00.000Z,"{'USD': {'price': 1.3112408441444283, 'volume_..."


# Collect historic prices
- Source: https://api.pro.coinbase.com

In [4]:
def collect_historic_prices(listings:pd.DataFrame, start_date:str, end_date:str)->pd.DataFrame:

    date_ranges = utils.date_range_split(
        start_date  = start_date,
        end_date    = end_date,
        freq        = 200
        )

    tickers = list(listings['symbol'].unique())
    frames = []

    for ticker in tqdm(tickers):
        for date_range in date_ranges:

            frame = parsing.get_historic_prices(
                url         = config.historic_prices_url, 
                ticker      = ticker, 
                start_date  = date_range[0], 
                end_date    = date_range[1]
                )
            
            frames.append(frame)
            time.sleep(.5)

    historic_prices = pd.concat(frames, ignore_index=True)
    historic_prices.drop_duplicates(ignore_index=True, inplace=True)
    return historic_prices

In [5]:
prices = collect_historic_prices(
    listings=listings,
    start_date=config.date_range[0],
    end_date=config.date_range[1]
    )

100%|██████████| 3/3 [00:10<00:00,  3.61s/it]


In [6]:
prices

,date,low,high,open,close,volume,symbol
0,2022-11-01,20326.550,20691.000,20489.550,20479.630,21147.009,BTC
1,2022-10-31,20237.000,20837.680,20629.210,20489.940,22222.736,BTC
2,2022-10-30,20523.920,20939.350,20822.380,20630.680,13737.388,BTC
3,2022-10-29,20561.200,21080.000,20597.340,20823.530,19822.751,BTC
4,2022-10-28,20000.000,20755.090,20295.770,20597.910,29581.752,BTC
...,...,...,...,...,...,...,...
2188,2023-12-11,0.999,1.000,1.000,1.000,256469946.850,USDT
2189,2023-12-10,1.000,1.000,1.000,1.000,115706157.430,USDT
2190,2023-12-09,1.000,1.000,1.000,1.000,162390879.680,USDT
2191,2023-12-08,0.997,1.000,1.000,1.000,248183686.380,USDT


In [ ]:
prices.to_csv('data/historic_crypto_prices.csv', index=False)

In [9]:
prices.groupby(by = ['symbol'])['date'].min()

symbol
BTC    2022-04-16
ETH    2022-04-16
USDT   2022-04-16
Name: date, dtype: datetime64[ns]

In [10]:
prices.groupby(by = ['symbol'])['date'].max()

symbol
BTC    2024-04-15
ETH    2024-04-15
USDT   2024-04-15
Name: date, dtype: datetime64[ns]

In [4]:
listings

In [2]:
# Coinbase coins
r = requests.get('https://api.exchange.coinbase.com/products')
coins = pd.DataFrame.from_records(r.json())
coins = coins[(coins['quote_currency'].isin(['USD'])) & (coins['status'] == 'online')]
coins

,id,base_currency,quote_currency,quote_increment,base_increment,display_name,min_market_funds,margin_enabled,post_only,limit_only,cancel_only,status,status_message,trading_disabled,fx_stablecoin,max_slippage_percentage,auction_mode,high_bid_limit_percentage
3,USDT-USD,USDT,USD,0.00001,0.01,USDT-USD,1,False,False,False,False,online,,False,True,0.01000000,False,0.03000000
4,MINA-USD,MINA,USD,0.001,0.001,MINA-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
8,AMP-USD,AMP,USD,0.00001,1,AMP-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
11,MKR-USD,MKR,USD,0.01,0.000001,MKR-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
13,MAGIC-USD,MAGIC,USD,0.0001,0.01,MAGIC-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,COVAL-USD,COVAL,USD,0.00001,1,COVAL-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
612,YFI-USD,YFI,USD,0.01,0.000001,YFI-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
614,ARB-USD,ARB,USD,0.0001,0.01,ARB-USD,1,False,False,False,False,online,,False,False,0.03000000,False,
619,COMP-USD,COMP,USD,0.01,0.001,COMP-USD,1,False,False,False,False,online,,False,False,0.03000000,False,


In [7]:
# Collect categories
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/categories'
headers = {'X-CMC_PRO_API_KEY' : 'a5b199a5-0641-4d03-8cf8-2df1e19e52ef'}
r = requests.get(url, headers=headers)
categories = pd.DataFrame(r.json()['data'])
categories['category_id'] = categories['id'].values
categories

,id,name,title,description,num_tokens,avg_price_change,market_cap,market_cap_change,volume,volume_change,last_updated,category_id
0,6617dd1bd0384836b9c7bdf3,TokenFi Launchpad,TokenFi Launchpad,TokenFi Launchpad,4,17.495,1697258491.150,9.882,229840527.421,14.349,2024-04-11T12:53:04.936Z,6617dd1bd0384836b9c7bdf3
1,6604f2405726cb6104472fdb,Presale Memes,Presale Memes,Presale Memes,3,-1.680,197907676.670,-5.094,93783877.534,-14.822,2024-03-28T04:30:07.129Z,6604f2405726cb6104472fdb
2,66039e3c6253043a6a580e65,Move VM (MVM),Move VM (MVM),Move VM (MVM),2,3.290,6102170727.570,3.310,362344399.952,-32.187,2024-03-27T04:19:33.033Z,66039e3c6253043a6a580e65
3,65fafbaccd69e615d550867e,Atomicals Ecosystem,Atomicals Ecosystem,Atomicals Ecosystem,8,5.019,0.000,0.000,1196364.025,1.318,2024-03-20T16:19:23.093Z,65fafbaccd69e615d550867e
4,65f23191e6c934565751ce16,DePIN,DePIN,DePIN,134,4.300,35519017647.610,3.635,1454951902.358,-0.164,2024-03-13T23:07:23.483Z,65f23191e6c934565751ce16
...,...,...,...,...,...,...,...,...,...,...,...,...
229,60308028d2088f200c58a005,BNB Chain Ecosystem,Top BNB Chain Tokens by Market Capitalization,BNB Chain dual-chain architecture will empower...,1997,1.999,263436963395.930,1.186,53144787814.504,1646.696,2023-08-09T13:59:47.270Z,60308028d2088f200c58a005
230,60291fa0db1be76c46298e83,NFTs & Collectibles,Top NFTs & Collectibles Tokens by Market Capit...,A NFT (non-fungible token) is a special crypto...,772,2.681,43473075340.940,1.649,2670081428.573,1441.944,2023-08-09T13:45:05.229Z,60291fa0db1be76c46298e83
231,601cf8d2d8fd860e4ea5d96f,Polkadot Ecosystem,Top Polkadot Tokens by Market Capitalization,Polkadot is an open-source sharding multichain...,109,2.492,26859771608.100,2.408,789030165.279,18.026,2023-08-09T14:00:10.601Z,601cf8d2d8fd860e4ea5d96f
232,5fb62da404d1dd4c73744883,Storage,Top Storage Tokens by Market Capitalization,In blockchain-based decentralized cloud storag...,53,4.295,17011885866.130,4.569,691672697.369,5.251,2023-08-09T13:48:02.585Z,5fb62da404d1dd4c73744883


In [9]:
selected_cats = categories.sort_values(by=['num_tokens'], ascending=False).head(100)

frames = []
for _id in tqdm(selected_cats['id'].unique()):
    url = f'https://pro-api.coinmarketcap.com/v1/cryptocurrency/category?id={_id}&limit=100'
    headers = {'X-CMC_PRO_API_KEY' : 'a5b199a5-0641-4d03-8cf8-2df1e19e52ef'}
    r = requests.get(url, headers=headers)
    try:
        frame = pd.DataFrame.from_records(r.json()['data']['coins'])
        frame['category_id'] = _id
        frames.append(frame)
        time.sleep(.5)
    except:
        continue

category_mapping = pd.concat(frames, ignore_index=True)
category_mapping = category_mapping[['id', 'symbol', 'category_id']]
category_mapping = category_mapping.merge(categories[['category_id', 'name', 'description']], how='left', on=['category_id'])
category_mapping

100%|██████████| 100/100 [01:16<00:00,  1.30it/s]
/var/folders/b2/17vn_90d5t93nhh5__zt6tb80000gn/T/ipykernel_37335/1230674035.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  category_mapping = pd.concat(frames, ignore_index=True)


,id,symbol,category_id,name,description
0,74,DOGE,6051a82566fc1b42617d6dc6,Memes,Memes
1,5994,SHIB,6051a82566fc1b42617d6dc6,Memes,Memes
2,28752,WIF,6051a82566fc1b42617d6dc6,Memes,Memes
3,24478,PEPE,6051a82566fc1b42617d6dc6,Memes,Memes
4,10804,FLOKI,6051a82566fc1b42617d6dc6,Memes,Memes
...,...,...,...,...,...
3596,2274,MDS,6051a80b66fc1b42617d6da4,Health,Health
3597,6237,TMED,6051a80b66fc1b42617d6da4,Health,Health
3598,1529,KUSH,6051a80b66fc1b42617d6da4,Health,Health
3599,2497,MTN,6051a80b66fc1b42617d6da4,Health,Health


In [10]:
category_mapping.to_csv('data/crypto_categories.csv', index=False)

In [15]:
l = sample.merge(category_mapping, how='left', on=['symbol'])
l

,id_x,base_currency,quote_currency,quote_increment,base_increment,display_name,min_market_funds,margin_enabled,post_only,limit_only,...,high_bid_limit_percentage,symbol,name_x,date_added,tags,market_cap,id_y,category_id,name_y,description
0,BTC-USD,BTC,USD,0.01,0.00000001,BTC-USD,1,False,False,False,...,,BTC,Bitcoin,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",1277315288632.126,NaN,NaN,NaN,NaN
1,ETH-USD,ETH,USD,0.01,0.00000001,ETH-USD,1,False,False,False,...,,ETH,Ethereum,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",377101665650.718,1027.000,618c0beeb7dd913155b462f9,Ethereum Ecosystem,Ethereum Ecosystem
2,ETH-USD,ETH,USD,0.01,0.00000001,ETH-USD,1,False,False,False,...,,ETH,Ethereum,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",377101665650.718,1027.000,604f2752ebccdd50cd175fc0,Smart Contracts,Smart Contracts
3,ETH-USD,ETH,USD,0.01,0.00000001,ETH-USD,1,False,False,False,...,,ETH,Ethereum,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",377101665650.718,1027.000,620a2bac3ee27777ad05bdaa,Injective Ecosystem,Injective Ecosystem
4,ETH-USD,ETH,USD,0.01,0.00000001,ETH-USD,1,False,False,False,...,,ETH,Ethereum,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",377101665650.718,1027.000,6433de7df79a2653906cd680,Layer 1,Layer 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,MASK-USD,MASK,USD,0.01,0.01,MASK-USD,1,False,False,False,...,,MASK,Mask Network,2021-02-21T00:00:00.000Z,"[communications-social-media, collectibles-nft...",345972306.196,8536.000,618c0beeb7dd913155b462f9,Ethereum Ecosystem,Ethereum Ecosystem
350,MASK-USD,MASK,USD,0.01,0.01,MASK-USD,1,False,False,False,...,,MASK,Mask Network,2021-02-21T00:00:00.000Z,"[communications-social-media, collectibles-nft...",345972306.196,8536.000,6053dfb66be1bf5c15e865ee,Metaverse,Metaverse
351,MASK-USD,MASK,USD,0.01,0.01,MASK-USD,1,False,False,False,...,,MASK,Mask Network,2021-02-21T00:00:00.000Z,"[communications-social-media, collectibles-nft...",345972306.196,8536.000,60a5f6765abd81761fe58688,Polygon Ecosystem,Polygon Ecosystem
352,MASK-USD,MASK,USD,0.01,0.01,MASK-USD,1,False,False,False,...,,MASK,Mask Network,2021-02-21T00:00:00.000Z,"[communications-social-media, collectibles-nft...",345972306.196,8536.000,648afd0f36e0e773f2c815c2,DWF Labs Portfolio,DWF Labs Portfolio


In [16]:
l[l['name_y'].isnull()]

,id_x,base_currency,quote_currency,quote_increment,base_increment,display_name,min_market_funds,margin_enabled,post_only,limit_only,...,high_bid_limit_percentage,symbol,name_x,date_added,tags,market_cap,id_y,category_id,name_y,description
0,BTC-USD,BTC,USD,0.01,0.00000001,BTC-USD,1,False,False,False,...,,BTC,Bitcoin,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",1277315288632.126,NaN,NaN,NaN,NaN
13,XRP-USD,XRP,USD,0.0001,0.000001,XRP-USD,1,False,False,False,...,,XRP,XRP,2013-08-04T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, arr...",28928065315.656,NaN,NaN,NaN,NaN
62,LTC-USD,LTC,USD,0.01,0.00000001,LTC-USD,1,False,False,False,...,,LTC,Litecoin,2013-04-28T00:00:00.000Z,"[mineable, pow, scrypt, medium-of-exchange]",6327768372.639,NaN,NaN,NaN,NaN
126,OP-USD,OP,USD,0.001,0.01,OP-USD,1,False,False,False,...,,OP,Optimism,2022-05-31T16:19:46.000Z,"[layer-2, optimism-ecosystem, modular-blockchain]",2584312720.137,NaN,NaN,NaN,NaN
148,QNT-USD,QNT,USD,0.01,0.001,QNT-USD,1,False,False,False,...,,QNT,Quant,2018-08-10T00:00:00.000Z,"[platform, interoperability]",1317697776.752,NaN,NaN,NaN,NaN
195,MINA-USD,MINA,USD,0.001,0.001,MINA-USD,1,False,False,False,...,,MINA,Mina,2021-03-02T00:00:00.000Z,"[pos, zero-knowledge-proofs, staking, coinbase...",938193757.252,NaN,NaN,NaN,NaN
344,T-USD,T,USD,0.00001,1,T-USD,1,False,False,False,...,,T,Threshold,2022-01-31T08:26:28.000Z,[bitcoin-ecosystem],352494847.332,NaN,NaN,NaN,NaN


In [14]:
category_mapping['name'].unique()


array(['Memes', 'BNB Chain Ecosystem', 'NFTs & Collectibles', 'DeFi',
       'Gaming', 'Ethereum Ecosystem', 'Solana Ecosystem', 'Play To Earn',
       'Doggone Doggerel', 'Avalanche Ecosystem', 'AI & Big Data',
       'Metaverse', 'Arbitrum Ecosystem', 'Polygon Ecosystem',
       'Yield Farming', 'DAO', 'Marketplace', 'DWF Labs Portfolio',
       'Smart Contracts', 'Decentralized Exchange (DEX) Token',
       'Governance', 'Stablecoin', 'Masternodes', 'Fantom Ecosystem',
       'Injective Ecosystem', 'Real World Assets', 'DePIN', 'Launchpad',
       'Distributed Computing', 'Entertainment', 'Sports', 'Layer 1',
       'Polkadot Ecosystem', 'Privacy', 'Asset Management',
       'Terra Ecosystem', 'HECO Ecosystem', 'Social Token', 'Health'],
      dtype=object)

In [ ]:
# res = coins.merge(volume[['id', 'rfq_volume_30day']], how='left', on=['id'])
# res.sort_values(by = ['rfq_volume_30day'], ascending=False).head(30)

In [ ]:
url = 'https://api.pro.coinbase.com/products/BTC-USD/candles?start=2023-08-01T12:00:00&end=2024-04-20T12:00:00&granularity=86400'
r = requests.get(url)
frame = pd.DataFrame(r.json(), columns = ['date', 'low', 'high', 'open', 'close', 'volume'])
frame['date'] = pd.to_datetime(frame['date'], unit='s')
frame

,date,low,high,open,close,volume
0,2024-04-20,63121.15,65463.08,63841.96,65058.44,4191.250006
1,2024-04-19,59573.32,65498.99,63514.84,63848.37,22838.079342
2,2024-04-18,60816.07,64210.00,61275.73,63513.85,15246.985398
3,2024-04-17,59648.33,64524.14,63814.94,61279.36,21811.867049
4,2024-04-16,61645.09,64400.00,63452.18,63814.94,17710.282132
...,...,...,...,...,...,...
258,2023-08-06,28955.00,29163.24,29047.33,29040.01,2461.640121
259,2023-08-05,28946.17,29111.87,29077.04,29047.12,2490.847105
260,2023-08-04,28747.10,29312.28,29174.84,29076.48,7291.146877
261,2023-08-03,28938.93,29399.97,29163.87,29174.18,8920.014777
